<h1> Data Wrangling of Electoral Data</h1>
<h2>Get our environment set up</h2>
The first thing we'll need to do is load in the libraries we'll be using.

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import re

from subprocess import check_output

<h2>Load Data</h2>


In [6]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

NA2 = pd.read_csv("data/National Assembly 2002 - Updated.csv", encoding = "ISO-8859-1")
NA8 = pd.read_csv("data/National Assembly 2008.csv", encoding = "ISO-8859-1")
NA13 = pd.read_csv("data/National Assembly 2013.csv", encoding = "ISO-8859-1")
print("Data Dimensions are: ", NA2.shape)
print("Data Dimensions are: ", NA8.shape)
print("Data Dimensions are: ", NA13.shape)

Data Dimensions are:  (1793, 11)
Data Dimensions are:  (2316, 11)
Data Dimensions are:  (4510, 11)


<h2>Data Info</h2>

Let's look into the info of provided dataset.

In [7]:
print("NA 2002.csv")
NA2.info()
print("\nNA 2008.csv")
NA8.info()
print("\nNA 2013.csv")
NA13.info()

NA 2002.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1793 entries, 0 to 1792
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   District                 1793 non-null   object 
 1   Seat                     1793 non-null   object 
 2   Constituency_title       1793 non-null   object 
 3   Candidate_Name           1793 non-null   object 
 4   Party                    1793 non-null   object 
 5   Votes                    1793 non-null   int64  
 6   Total_Valid_Votes        1793 non-null   int64  
 7   Total_Rejected_Votes     1793 non-null   int64  
 8   Total_Votes              1793 non-null   int64  
 9   Total_Registered_Voters  1793 non-null   int64  
 10  Turnout                  1793 non-null   float64
dtypes: float64(1), int64(5), object(5)
memory usage: 154.2+ KB

NA 2008.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2316 entries, 0 to 2315
Data columns (total 11 colum

All three files have 11 columns. Row details are as follow:

* NA 2002: 1,792
* NA 2008: 2,315
* NA 2013: 4,541

NA 2002.cv file has fine data types. 

In NA 2008 & NA 2013 files:
* 1st column name is missing and showing that it have int64 data type. 
* Turnout column has read as object.

<h2>Data Wrangling </h2>
Lets observe the 1st file in order to fix next two and merge them all into one dataframe.

In [8]:
print(NA2.head())
print(NA8.head())
print(NA13.head())
print(NA8.columns, "\n>>\n", NA13.columns)

   District        Seat Constituency_title  \
0  PESHAWAR  PESHAWAR-I               NA-1   
1  PESHAWAR  PESHAWAR-I               NA-1   
2  PESHAWAR  PESHAWAR-I               NA-1   
3  PESHAWAR  PESHAWAR-I               NA-1   
4  PESHAWAR  PESHAWAR-I               NA-1   

                       Candidate_Name                              Party  \
0                   Mr Sajid Abdullah           Pakistan Tehreek-e-Insaf   
1                     Mr Shabir Ahmad  Muttahidda Majlis-e-Amal Pakistan   
2              Mr Usman Bashir Bilour               Awami National Party   
3  Mr Muhammad Khurshid Khan Advocate                        Independent   
4            Mr Muhammad Muazzam Butt         Pakistan Muslim League(QA)   

   Votes  Total_Valid_Votes  Total_Rejected_Votes  Total_Votes  \
0   2029              65642                  1552        67194   
1  37179              65642                  1552        67194   
2  23002              65642                  1552        67194   
3 

So the first column should be District. We will extract district names from Seat column.
We will drop last column from NA13 because it contain no value..

<b>Rename Column and Replace Values </b>

In [9]:

NA8.rename(columns={'Unnamed: 0':'District'}, inplace=True)
NA13.rename(columns={'Unnamed: 0':'District'}, inplace=True)
print("NA 8: ", NA8.columns, "\nNA 13: ", NA13.columns)
#NA13 = NA13.drop('Unnamed: 11', axis=1)

NA 8:  Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout'],
      dtype='object') 
NA 13:  Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout'],
      dtype='object')


In [10]:
NA8.District = NA8.Seat

In [11]:
NA8['District'] = NA8['District'].str.replace("."," ") # to deal with D.I. Khan
NA8.District.unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1769182689.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NA8['District'] = NA8['District'].str.replace("."," ") # to deal with D.I. Khan


array(['Peshawar-1', 'Peshawar-II', 'Peshawar-III', 'Peshawar-IV',
       'Nowshera-I', 'Nowshera-II', 'Charsadda-I', 'Mardan-I',
       'Mardan-II', 'Mardan-III', 'Swabi-I', 'Swabi-II', 'Kohat', 'Karak',
       'Hangu', 'Abbottabad-I', 'Abbottabad-II', 'Haripur', 'Mansehra-I',
       'Mansehra-II', 'Battagram', 'Kohistan', 'D I Khan',
       'D I Khan-com-Tank', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat-I',
       'Swat-II', 'Shangla', 'Chitral',
       'Upper Dir-cum-Lower Dir (Old Upper Dir', 'Lower Dir',
       'Malakand P A', 'Tribal Area-I', 'Tribal Area-II',
       'Tribal Area-III', 'Tribal Area-IV', 'Tribal Area-V',
       'Tribal Area-VI', 'Tribal Area-VII', 'Tribal Area-VIII',
       'Tribal Area-IX', 'Tribal Area-X', 'Tribal Area-XI',
       'Tribal Area-XII', 'Islamabad-I', 'Islamabad-II', 'Rawalpindi-I',
       'Rawalpindi -II', 'Rawalpindi-III', 'Rawalpindi-IV',
       'Rawalpindi \x96V', 'Rawalpindi-VI', 'Rawalpindi-VII', 'Attock-I',
       'Attock-II', 'Attock-III', 'Cha

In [12]:
# remove all those substring with () 
    # Search for opening bracket in the name followed by (explained more in the practice data)
    # .* means zero or more instances of whatever precedes it
NA8['District'] = NA8['District'].str.replace(r"\(.*?","")
NA8.District.unique()
# OR

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1912675360.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  NA8['District'] = NA8['District'].str.replace(r"\(.*?","")


array(['Peshawar-1', 'Peshawar-II', 'Peshawar-III', 'Peshawar-IV',
       'Nowshera-I', 'Nowshera-II', 'Charsadda-I', 'Mardan-I',
       'Mardan-II', 'Mardan-III', 'Swabi-I', 'Swabi-II', 'Kohat', 'Karak',
       'Hangu', 'Abbottabad-I', 'Abbottabad-II', 'Haripur', 'Mansehra-I',
       'Mansehra-II', 'Battagram', 'Kohistan', 'D I Khan',
       'D I Khan-com-Tank', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat-I',
       'Swat-II', 'Shangla', 'Chitral',
       'Upper Dir-cum-Lower Dir Old Upper Dir', 'Lower Dir',
       'Malakand P A', 'Tribal Area-I', 'Tribal Area-II',
       'Tribal Area-III', 'Tribal Area-IV', 'Tribal Area-V',
       'Tribal Area-VI', 'Tribal Area-VII', 'Tribal Area-VIII',
       'Tribal Area-IX', 'Tribal Area-X', 'Tribal Area-XI',
       'Tribal Area-XII', 'Islamabad-I', 'Islamabad-II', 'Rawalpindi-I',
       'Rawalpindi -II', 'Rawalpindi-III', 'Rawalpindi-IV',
       'Rawalpindi \x96V', 'Rawalpindi-VI', 'Rawalpindi-VII', 'Attock-I',
       'Attock-II', 'Attock-III', 'Chak

In [15]:
def remove_bracket_substrings(text):
#     pattern = r"\(.*?\)"  # Regex pattern to match substrings within round brackets
    pattern = r"\(.*?"  # Regex pattern to match substrings with openning bracket and instances preceding it

    return re.sub(pattern, "", text)
                  
# Apply the function to the 'Text' column of the DataFrame
NA8['District'] = NA8['District'].apply(remove_bracket_substrings)
NA8.District.unique()

array(['Peshawar-1', 'Peshawar-II', 'Peshawar-III', 'Peshawar-IV',
       'Nowshera-I', 'Nowshera-II', 'Charsadda-I', 'Mardan-I',
       'Mardan-II', 'Mardan-III', 'Swabi-I', 'Swabi-II', 'Kohat', 'Karak',
       'Hangu', 'Abbottabad-I', 'Abbottabad-II', 'Haripur', 'Mansehra-I',
       'Mansehra-II', 'Battagram', 'Kohistan', 'D I Khan',
       'D I Khan-com-Tank', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat-I',
       'Swat-II', 'Shangla', 'Chitral',
       'Upper Dir-cum-Lower Dir Old Upper Dir', 'Lower Dir',
       'Malakand P A', 'Tribal Area-I', 'Tribal Area-II',
       'Tribal Area-III', 'Tribal Area-IV', 'Tribal Area-V',
       'Tribal Area-VI', 'Tribal Area-VII', 'Tribal Area-VIII',
       'Tribal Area-IX', 'Tribal Area-X', 'Tribal Area-XI',
       'Tribal Area-XII', 'Islamabad-I', 'Islamabad-II', 'Rawalpindi-I',
       'Rawalpindi -II', 'Rawalpindi-III', 'Rawalpindi-IV',
       'Rawalpindi \x96V', 'Rawalpindi-VI', 'Rawalpindi-VII', 'Attock-I',
       'Attock-II', 'Attock-III', 'Chak

In [16]:
# remove numeric 
# [a-zA-Z] means any a-z or A-Z at the start of a line [^a-zA-Z] means any character that IS NOT a-z OR A-Z
NA8['District']  = NA8['District'] .str.replace('[^a-zA-Z -]', '')
NA8.District.unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\15992113.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  NA8['District']  = NA8['District'] .str.replace('[^a-zA-Z -]', '')


array(['Peshawar-', 'Peshawar-II', 'Peshawar-III', 'Peshawar-IV',
       'Nowshera-I', 'Nowshera-II', 'Charsadda-I', 'Mardan-I',
       'Mardan-II', 'Mardan-III', 'Swabi-I', 'Swabi-II', 'Kohat', 'Karak',
       'Hangu', 'Abbottabad-I', 'Abbottabad-II', 'Haripur', 'Mansehra-I',
       'Mansehra-II', 'Battagram', 'Kohistan', 'D I Khan',
       'D I Khan-com-Tank', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat-I',
       'Swat-II', 'Shangla', 'Chitral',
       'Upper Dir-cum-Lower Dir Old Upper Dir', 'Lower Dir',
       'Malakand P A', 'Tribal Area-I', 'Tribal Area-II',
       'Tribal Area-III', 'Tribal Area-IV', 'Tribal Area-V',
       'Tribal Area-VI', 'Tribal Area-VII', 'Tribal Area-VIII',
       'Tribal Area-IX', 'Tribal Area-X', 'Tribal Area-XI',
       'Tribal Area-XII', 'Islamabad-I', 'Islamabad-II', 'Rawalpindi-I',
       'Rawalpindi -II', 'Rawalpindi-III', 'Rawalpindi-IV',
       'Rawalpindi V', 'Rawalpindi-VI', 'Rawalpindi-VII', 'Attock-I',
       'Attock-II', 'Attock-III', 'Chakwal-I

https://stackoverflow.com/questions/33022051/regex-explanation#:~:text=%40CharlieParker%20%5E%20is%20beginning%20of%20the%20line%20and%20%24%20is%20end%20of%20the%20line.%20So%20if%20you%20have%20pattern%3A%20xyz%24%2C%20it%20will%20match%20the%20value%3A%20abcxyz.%20However%20pattern%3A%20%5Exyz%24%20will%20not%20match.%20Then%20pattern%3A%20%5Eabc%20will%20also%20match%20value%3A%20abcxyz%20and%20pattern%3A%20%5Eabc%24%20will%20not.

In [17]:
NA8['District']  = NA8['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
# NA8['District']  = NA8['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA8.District.unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\352808575.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  NA8['District']  = NA8['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')


array(['Peshawar-', 'Peshawar-II', 'Peshawar-III', 'Peshawar-IV',
       'Nowshera-I', 'Nowshera-II', 'Charsadda-I', 'Mardan-I',
       'Mardan-II', 'Mardan-III', 'Swabi-I', 'Swabi-II', 'Kohat', 'Karak',
       'Hangu', 'Abbottabad-I', 'Abbottabad-II', 'Haripur', 'Mansehra-I',
       'Mansehra-II', 'Battagram', 'Kohistan', 'D I Khan',
       'D I Khan-com-Tank', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat-I',
       'Swat-II', 'Shangla', 'Chitral',
       'Upper Dir-cum-Lower Dir Old Upper Dir', 'Lower Dir',
       'Malakand P A', 'Tribal Area-I', 'Tribal Area-II',
       'Tribal Area-III', 'Tribal Area-IV', 'Tribal Area-V',
       'Tribal Area-VI', 'Tribal Area-VII', 'Tribal Area-VIII',
       'Tribal Area-IX', 'Tribal Area-X', 'Tribal Area-XI',
       'Tribal Area-XII', 'Islamabad-I', 'Islamabad-II', 'Rawalpindi-I',
       'Rawalpindi -II', 'Rawalpindi-III', 'Rawalpindi-IV', 'Rawalpindi',
       'Rawalpindi-VI', 'Rawalpindi-VII', 'Attock-I', 'Attock-II',
       'Attock-III', 'Chakwal-I',

In [18]:
NA8.District = NA8.Seat#.str.split("-", expand=True)[0]
#Add District column
#NA8['District'] = NA8['Seat']
NA8['District'] = NA8['District'].str.replace("."," ") # to deal with D.I. Khan
# remove all those substring with () 
NA8['District'] = NA8['District'].str.replace(r"\(.*\)","")
# remove numeric
NA8['District']  = NA8['District'] .str.replace('[^a-zA-Z -]', '')
#NA8['District'] = NA8['District'].str.replace(r"Cum.*","")
#NA8['District'] = NA8['District'].str.replace(r"cum.*","")
#na18['District'] = na18['District'].str.replace(r"KUM.*","")
# to convert Tribal Area III - Mohman into Tribal Area III
NA8['District'] = NA8['District'].str.replace(r"-.*","")
NA8['District']  = NA8['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA8['District']  = NA8['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA8['District'].unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\2539223980.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NA8['District'] = NA8['District'].str.replace("."," ") # to deal with D.I. Khan
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\2539223980.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  NA8['District'] = NA8['District'].str.replace(r"\(.*\)","")
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\2539223980.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  NA8['District']  = NA8['District'] .str.replace('[^a-zA-Z -]', '')
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\2539223980.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  NA8['District'] = NA8['District'].str.replace(r"

array(['Peshawar', 'Nowshera', 'Charsadda', 'Mardan', 'Swabi', 'Kohat',
       'Karak', 'Hangu', 'Abbottabad', 'Haripur', 'Mansehra', 'Battagram',
       'Kohistan', 'D I Khan', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat',
       'Shangla', 'Chitral', 'Upper Dir', 'Lower Dir', 'Malakand P A',
       'Tribal Area', 'Islamabad', 'Rawalpindi', 'Attock', 'Chakwal',
       'Jhelum', 'Sargodha', 'Khushab', 'Mianwali', 'Bhakkar',
       'Faisalabad', 'Jhang', 'Toba Tek Singh', 'Gujranwala', 'Hafizabad',
       'Gujrat', 'M B Din', 'Sialkot', 'Narowal', 'Lahore', 'Sheikhupura',
       'Nanka Sahib', 'Nankana Sahib', 'Kasur', 'Okara', 'Multan',
       'Lodhran', 'Khanewal', 'Sahiwal', 'Pakpattan', 'Vehari',
       'Dera Ghazi Khan', 'Rajanpur', 'Muzaffargarh', 'Layyah',
       'Bahawalpur', 'Bahawalnagar', 'Rahimyar Khan',
       'Sukkur cum ShikarpurI', 'Sukkur cum ShikarpurII', 'Ghotki',
       'Shikarpur Old Shikarpur', 'Shikarpur', 'Larkana Old Larkana',
       'Larka cum', 'Kamber Shahdadkot 

In [19]:
NA13.District = NA13.Seat #.str.split("-", expand=True)[0]
#Add District column
#NA13['District'] = NA13['Seat']
NA13['District'] = NA8['District'].str.replace("."," ") # to deal with D.I. Khan
# remove all those substring with () 
NA13['District'] = NA13['District'].str.replace(r"\(.*\)","")
# remove numeric
NA13['District']  = NA13['District'] .str.replace('[^a-zA-Z -]', '')
NA13['District'] = NA13['District'].str.replace(r"Cum.*","")
#na18['District'] = na18['Distirct'].str.replace(r"KUM.*","")
# to convert Tribal Area III - Mohman into Tribal Area III
NA13['District'] = NA13['District'].str.replace(r"-.*","")
NA13['District']  = NA13['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA13['District']  = NA13['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA13['District'].unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\3229997259.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NA13['District'] = NA8['District'].str.replace("."," ") # to deal with D.I. Khan
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\3229997259.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  NA13['District'] = NA13['District'].str.replace(r"\(.*\)","")
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\3229997259.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  NA13['District']  = NA13['District'] .str.replace('[^a-zA-Z -]', '')
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\3229997259.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  NA13['District'] = NA13['District'].str.repl

array(['Peshawar', 'Nowshera', 'Charsadda', 'Mardan', 'Swabi', 'Kohat',
       'Karak', 'Hangu', 'Abbottabad', 'Haripur', 'Mansehra', 'Battagram',
       'Kohistan', 'D I Khan', 'Bannu', 'Lakki Marwat', 'Buner', 'Swat',
       'Shangla', 'Chitral', 'Upper Dir', 'Lower Dir', 'Malakand P A',
       'Tribal Area', 'Islamabad', 'Rawalpindi', 'Attock', 'Chakwal',
       'Jhelum', 'Sargodha', 'Khushab', 'Mianwali', 'Bhakkar',
       'Faisalabad', 'Jhang', 'Toba Tek Singh', 'Gujranwala', 'Hafizabad',
       'Gujrat', 'M B Din', 'Sialkot', 'Narowal', 'Lahore', 'Sheikhupura',
       'Nanka Sahib', 'Nankana Sahib', 'Kasur', 'Okara', 'Multan',
       'Lodhran', 'Khanewal', 'Sahiwal', 'Pakpattan', 'Vehari',
       'Dera Ghazi Khan', 'Rajanpur', 'Muzaffargarh', 'Layyah',
       'Bahawalpur', 'Bahawalnagar', 'Rahimyar Khan',
       'Sukkur cum ShikarpurI', 'Sukkur cum ShikarpurII', 'Ghotki',
       'Shikarpur Old Shikarpur', 'Shikarpur', 'Larkana Old Larkana',
       'Larka cum', 'Kamber Shahdadkot 

In [20]:
NA13.head()

,District,Seat,ConstituencyTitle,CandidateName,Party,Votes,TotalValidVotes,TotalRejectedVotes,TotalVotes,TotalRegisteredVoters,Turnout
0,Peshawar,PESHAWAR-I,NA-1,Aamir Shehzad Hashmi,Mustaqbil Pakistan,77,145924,2103,146044,320578,46.18%
1,Peshawar,PESHAWAR-I,NA-1,Akram Khan,Independent,182,145924,2103,146044,320578,46.18 %
2,Peshawar,PESHAWAR-I,NA-1,Alhaaj Ghulam Ahmad Bilour,Awami National Party,24468,145924,2103,146044,320578,46.18 %
3,Peshawar,PESHAWAR-I,NA-1,Amir Syed,Pakistan Peoples Party (Shaheed Bhutto),454,145924,2103,146044,320578,46.18 %
4,Peshawar,PESHAWAR-I,NA-1,Bashir Ahmad Afridi,Muttahida Qaumi Movement Pakistan,117,145924,2103,146044,320578,46.18 %


We are all set with first issue. Turnout column has % symbol in it which makes it object datatype. We will remove non-numeric characters and change datatype.

<b>Change the datatype of Turnout column </b>

In [21]:
#  rstrip - Remove the trailing characters
NA8['Turnout'] = NA8['Turnout'].str.rstrip('%')
NA13['Turnout'] = NA13['Turnout'].str.rstrip('%')
NA8['Turnout'] = pd.to_numeric(NA8['Turnout'], errors='coerce')
NA13['Turnout'] = pd.to_numeric(NA13['Turnout'], errors='coerce')
NA8['Turnout'].head()

0    22.97
1    22.97
2    22.97
3    22.97
4    22.97
Name: Turnout, dtype: float64

Now our dataset is aligned and ready to merge. But before merging, lets add another column  'Year'. 

In [22]:
NA2['Year'] = "2002"
NA8['Year'] = "2008"
NA13['Year'] = "2013"

In [23]:
print(NA2.head(), "\n", NA8.head(), "\n", NA13.head())

   District        Seat Constituency_title  \
0  PESHAWAR  PESHAWAR-I               NA-1   
1  PESHAWAR  PESHAWAR-I               NA-1   
2  PESHAWAR  PESHAWAR-I               NA-1   
3  PESHAWAR  PESHAWAR-I               NA-1   
4  PESHAWAR  PESHAWAR-I               NA-1   

                       Candidate_Name                              Party  \
0                   Mr Sajid Abdullah           Pakistan Tehreek-e-Insaf   
1                     Mr Shabir Ahmad  Muttahidda Majlis-e-Amal Pakistan   
2              Mr Usman Bashir Bilour               Awami National Party   
3  Mr Muhammad Khurshid Khan Advocate                        Independent   
4            Mr Muhammad Muazzam Butt         Pakistan Muslim League(QA)   

   Votes  Total_Valid_Votes  Total_Rejected_Votes  Total_Votes  \
0   2029              65642                  1552        67194   
1  37179              65642                  1552        67194   
2  23002              65642                  1552        67194   
3 

<h2>NAN Values</h2>
Lets check the status of NA values

In [24]:
print("NA2", NA2.isnull().any(), "\nNA8: ", NA8.isnull().any(), "\nNA13:", NA13.isnull().any())

NA2 District                   False
Seat                       False
Constituency_title         False
Candidate_Name             False
Party                      False
Votes                      False
Total_Valid_Votes          False
Total_Rejected_Votes       False
Total_Votes                False
Total_Registered_Voters    False
Turnout                    False
Year                       False
dtype: bool 
NA8:  District                 False
Seat                     False
ConstituencyTitle        False
CandidateName            False
Party                    False
Votes                    False
TotalValidVotes          False
TotalRejectedVotes       False
TotalVotes               False
TotalRegisteredVoters    False
Turnout                  False
Year                     False
dtype: bool 
NA13: District                  True
Seat                     False
ConstituencyTitle        False
CandidateName            False
Party                    False
Votes                    False
Tota

There is no null record.

Final step before merging:

Just to confirm that column names are similar in all files.

In [25]:
print("\n NA2", NA2.columns, "\n NA8", NA8.columns, "\n NA13", NA13.columns)


 NA2 Index(['District', 'Seat', 'Constituency_title', 'Candidate_Name', 'Party',
       'Votes', 'Total_Valid_Votes', 'Total_Rejected_Votes', 'Total_Votes',
       'Total_Registered_Voters', 'Turnout', 'Year'],
      dtype='object') 
 NA8 Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout', 'Year'],
      dtype='object') 
 NA13 Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout', 'Year'],
      dtype='object')


In [26]:
NA2.rename(columns={'Constituency_title':'ConstituencyTitle', 'Candidate_Name':'CandidateName', 'Total_Valid_Votes':'TotalValidVotes', 'Total_Rejected_Votes':'TotalRejectedVotes', 'Total_Votes':'TotalVotes', 'Total_Registered_Voters':'TotalRegisteredVoters', }, inplace=True)
NA2.columns

Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout', 'Year'],
      dtype='object')

<h2>Concatenate All 3 Datasets </h3>

In [27]:
df = pd.concat([NA2, NA8, NA13])
df.shape
df.head()

,District,Seat,ConstituencyTitle,CandidateName,Party,Votes,TotalValidVotes,TotalRejectedVotes,TotalVotes,TotalRegisteredVoters,Turnout,Year
0,PESHAWAR,PESHAWAR-I,NA-1,Mr Sajid Abdullah,Pakistan Tehreek-e-Insaf,2029,65642,1552,67194,233907,28.73,2002
1,PESHAWAR,PESHAWAR-I,NA-1,Mr Shabir Ahmad,Muttahidda Majlis-e-Amal Pakistan,37179,65642,1552,67194,233907,28.73,2002
2,PESHAWAR,PESHAWAR-I,NA-1,Mr Usman Bashir Bilour,Awami National Party,23002,65642,1552,67194,233907,28.73,2002
3,PESHAWAR,PESHAWAR-I,NA-1,Mr Muhammad Khurshid Khan Advocate,Independent,1537,65642,1552,67194,233907,28.73,2002
4,PESHAWAR,PESHAWAR-I,NA-1,Mr Muhammad Muazzam Butt,Pakistan Muslim League(QA),1417,65642,1552,67194,233907,28.73,2002


In [28]:
df.isnull().any()

District                  True
Seat                     False
ConstituencyTitle        False
CandidateName            False
Party                    False
Votes                    False
TotalValidVotes          False
TotalRejectedVotes       False
TotalVotes               False
TotalRegisteredVoters    False
Turnout                  False
Year                     False
dtype: bool

<h2>Some Preliminary Text Pre-processing</h2>
Here, I'm interested in cleaning up all Text columns to make sure there's no data entry inconsistencies in it. We could go through and check each row by hand, of course, and hand-correct inconsistencies when we find them. There's a more efficient way to do this though!

In [29]:
# get all the unique values in the 'District' column
#df['District'] = df['District'].astype(str)
dist = df['District'].unique()
#dist.sort()
dist

array(['PESHAWAR', 'NOWSHERA', 'Nowshera', 'Charsadda', 'Charsdda',
       'Mardan', 'Sawabi', 'Kohat', 'Karak', 'Hangu', 'Abbottabad',
       'Haripur', 'Mansehra', 'Batagram', 'Kohistan', 'Dera Ismail Khan',
       'Dera Ismail Khan cum Tank', 'Bannu', 'Lakki Marwat', 'Buner',
       'Sawat', 'Shangla', 'Chitral', 'Upper Dir', 'Lower Dir',
       'Malakand', 'Tribal Area', 'Islamabad', 'Rawalpindi', 'Attock',
       'Chakwal', 'Jhelum', 'Sargodha', 'Khushab', 'khushab', 'Mianwali',
       'Bhakkar', 'Faisalabad', 'Jhang', 'Toba Tek Singh', 'Gujranwala',
       'Hafizabad', 'Gujrat', 'Mandi Bahauddin', 'Sialkot', 'Narowal',
       'Lahore', 'Sheikhupura', 'Sheiklhupura', 'Kasur', 'Okara',
       'Multan', 'Lodhran', 'Khanewal', 'Sahiwal', 'Pakpattan', 'Vehari',
       'Dera Ghazi Khan', 'Rajanpur', 'Muzaffargarh', 'Layyah',
       'Bahawalpur', 'Bahawalnagar', 'Rahimyar Khan', 'SUKKUR', 'Ghotki',
       'Shikarpur', 'Shikarapur', 'Larkana', 'Jacobabad',
       'Nausheroferoze', 'Nawab

Just looking at this, We can see some problems due to inconsistent data entry: 'PESHAWAR' and Peshawar ', for example, or 'Charsadda' and 'Charsdda'.

The first thing we are going to do is make everything lower case (we can change it back at the end if need) and remove any white spaces at the beginning and end of cells. Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this.

In [30]:
# convert to lower case
df['District'] = df['District'].str.lower()
# remove trailing white spaces
df['District'] = df['District'].str.strip()

<h2>Use fuzzy matching to correct inconsistent data entry</h2>
Alright, let's take another look at the district column and see if there's any more data cleaning we need to do.

In [31]:
dist = df['District'].unique()
#dist.sort()
dist

array(['peshawar', 'nowshera', 'charsadda', 'charsdda', 'mardan',
       'sawabi', 'kohat', 'karak', 'hangu', 'abbottabad', 'haripur',
       'mansehra', 'batagram', 'kohistan', 'dera ismail khan',
       'dera ismail khan cum tank', 'bannu', 'lakki marwat', 'buner',
       'sawat', 'shangla', 'chitral', 'upper dir', 'lower dir',
       'malakand', 'tribal area', 'islamabad', 'rawalpindi', 'attock',
       'chakwal', 'jhelum', 'sargodha', 'khushab', 'mianwali', 'bhakkar',
       'faisalabad', 'jhang', 'toba tek singh', 'gujranwala', 'hafizabad',
       'gujrat', 'mandi bahauddin', 'sialkot', 'narowal', 'lahore',
       'sheikhupura', 'sheiklhupura', 'kasur', 'okara', 'multan',
       'lodhran', 'khanewal', 'sahiwal', 'pakpattan', 'vehari',
       'dera ghazi khan', 'rajanpur', 'muzaffargarh', 'layyah',
       'bahawalpur', 'bahawalnagar', 'rahimyar khan', 'sukkur', 'ghotki',
       'shikarpur', 'shikarapur', 'larkana', 'jacobabad',
       'nausheroferoze', 'nawabshah', 'khairpur', 'hyd

It does look like there are some remaining inconsistencies: 'charsadda' and 'charsdda' should probably be the same. 

We are going to use the fuzzywuzzy package to help identify which string are closest to each other. 
> <b>Fuzzy matching:</b> The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of districts that have the closest distance to "charsadda".

In [32]:
# get the top 10 closest matches to "charsadda"
matches = fuzzywuzzy.process.extract("charsadda", dist, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('charsadda', 100),
 ('charsdda', 94),
 ('mardan', 53),
 ('tharparkar', 53),
 ('kharan', 53),
 ('chakwal', 50),
 ('larkana', 50),
 ('rahimyar khan', 45),
 ('hafizabad', 44),
 ('jacobabad', 44)]

We can see that two of the items in the districts are very close to "charsadda": 'charsadda; and 'charsdda'.

Let's replace all rows in our District column that have a ratio of > 90 with "charsadda".

For the reusability,  I'm going to write a function to fix all these challenges ASAP.

In [33]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match

To test the funtion

In [34]:
# use the function we just wrote to replace close matches to "charsadda" 
replace_matches_in_column(df=df, column='District', string_to_match="charsadda")

In [35]:
# OR
# use the function we just wrote to replace close matches to "charsadda" 
replace_matches_in_column(df=df, column='District', string_to_match="charsadda")

In [36]:
dist = df['District'].unique()
#dist.sort()
dist

array(['peshawar', 'nowshera', 'charsadda', 'mardan', 'sawabi', 'kohat',
       'karak', 'hangu', 'abbottabad', 'haripur', 'mansehra', 'batagram',
       'kohistan', 'dera ismail khan', 'dera ismail khan cum tank',
       'bannu', 'lakki marwat', 'buner', 'sawat', 'shangla', 'chitral',
       'upper dir', 'lower dir', 'malakand', 'tribal area', 'islamabad',
       'rawalpindi', 'attock', 'chakwal', 'jhelum', 'sargodha', 'khushab',
       'mianwali', 'bhakkar', 'faisalabad', 'jhang', 'toba tek singh',
       'gujranwala', 'hafizabad', 'gujrat', 'mandi bahauddin', 'sialkot',
       'narowal', 'lahore', 'sheikhupura', 'sheiklhupura', 'kasur',
       'okara', 'multan', 'lodhran', 'khanewal', 'sahiwal', 'pakpattan',
       'vehari', 'dera ghazi khan', 'rajanpur', 'muzaffargarh', 'layyah',
       'bahawalpur', 'bahawalnagar', 'rahimyar khan', 'sukkur', 'ghotki',
       'shikarpur', 'shikarapur', 'larkana', 'jacobabad',
       'nausheroferoze', 'nawabshah', 'khairpur', 'hyderabad', 'badin',
 

Lets fix few more

In [37]:
replace_matches_in_column(df=df, column='District', string_to_match="nowshera")
replace_matches_in_column(df=df, column='District', string_to_match="rawalpindi")
replace_matches_in_column(df=df, column='District', string_to_match="sheikhupura")
replace_matches_in_column(df=df, column='District', string_to_match="shikarpur")
replace_matches_in_column(df=df, column='District', string_to_match="nankana sahib")

<h3>Lets Clean data around Party & Candidates Name </h3>

In [38]:
del dist

pty = df['Party'].unique()
pty.sort()
pty

array(['Aap Janab Sarkar Party', 'Afghan Qumi Movement (Pakistan)',
       'All Pakistan Bayrozgar Party', 'All Pakistan Muslim League',
       'All Pakistan Youth Working Party',
       'Awami Himayat Tehreek Pakistan', 'Awami Jamhuri Ittehad Pakistan',
       'Awami Justice Party', 'Awami Muslim League Pakistan',
       'Awami National Party', 'Awami Qaidat Party', 'Awami Qiadat Party',
       'Awami Workers Party', 'Azad Pakistan Party',
       'Bahawalpur National Awami Party', 'Balochistan National Congress',
       'Balochistan National Democratic Party',
       'Balochistan National Movement', 'Balochistan National Party',
       'Balochistan National Party (Awami)',
       'Balohistan National Movement', 'Christian Progressive Movement',
       'Communist Party of Pakistan', 'Ghareeb Awam Party',
       'Hazara Awami Ittehad Pakistan', 'Hazara Democratic Party',
       'Hazara Quami Mahaz', 'IndeIndependentdent',
       'IndeIndependentdentE', 'Independent', 'Independent (RETIR

In [39]:
df['Party'] = df['Party'].replace(['MUTTHIDA\xa0MAJLIS-E-AMAL\xa0PAKISTAN'], 'Muttahidda Majlis-e-Amal Pakistan')
df['Party'] = df['Party'].replace(['Pakistan Muslim League'], 'Pakistan Muslim League (QA)')
#converting text to lower case & removing white spaces
df['Party'] = df['Party'].str.lower()
df['Party'] = df['Party'].str.strip()

In [40]:
# As I coded this earlier, I wouldn't change it due to lower case letters. 
replace_matches_in_column(df=df, column='Party', string_to_match="Balochistan National Movement")
replace_matches_in_column(df=df, column='Party', string_to_match="Independent")
replace_matches_in_column(df=df, column='Party', string_to_match="Istiqlal Party")
replace_matches_in_column(df=df, column='Party', string_to_match="Jamote Qaumi Movement")
replace_matches_in_column(df=df, column='Party', string_to_match="Labour Party Pakistan")
replace_matches_in_column(df=df, column='Party', string_to_match="Mohib-e-Wattan Nowjawan Inqilabion Ki Anjuman (MNAKA)")
replace_matches_in_column(df=df, column='Party', string_to_match="Muttahida Qaumi Movement") # Muttahida Qaumi Movement Pakistan
replace_matches_in_column(df=df, column='Party', string_to_match="Muttahidda Majlis-e-Amal") # Muttahidda Majlis-e-Amal Pakistan
replace_matches_in_column(df=df, column='Party', string_to_match="National Peoples Party")
replace_matches_in_column(df=df, column='Party', string_to_match="Nizam-e-Mustafa Party")
replace_matches_in_column(df=df, column='Party', string_to_match="Pak Muslim Alliance")
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Awami Party")
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Democratic Party")
# After analyzing each of the below strings.
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Muslim League (QA)", min_ratio =97)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Muslim League (N)", min_ratio =97)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Muslim League (J)", min_ratio =97)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Muslim League (F)", min_ratio =97)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Peoples Party Parliamentarians", min_ratio =97)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Peoples Party(Shaheed Bhutto)", min_ratio =95)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Peoples Party(Sherpao)", min_ratio =97)
replace_matches_in_column(df=df, column='Party', string_to_match="Pakistan Tehreek-e-Insaf", min_ratio =95)
replace_matches_in_column(df=df, column='Party', string_to_match="Saraiki Sooba Movement Pakistan", min_ratio =95)


In [41]:
#fuzzywuzzy.process.extract("Pakistan Muslim League (QA)", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >97
#fuzzywuzzy.process.extract("Pakistan Muslim League (N)", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >97
#fuzzywuzzy.process.extract("Pakistan Muslim League (J)", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >97
#fuzzywuzzy.process.extract("Pakistan Muslim League (F)", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >97
#fuzzywuzzy.process.extract("Pakistan Peoples Party Parliamentarians", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >97
#fuzzywuzzy.process.extract("Pakistan Peoples Party(Shaheed Bhutto)", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >95
#fuzzywuzzy.process.extract("Pakistan Peoples Party(Sherpao)", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >97
#fuzzywuzzy.process.extract("Pakistan Tehreek-e-Insaf", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >95
#fuzzywuzzy.process.extract("Saraiki Sooba Movement Pakistan", pty, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >95


In [42]:
df['Party'] = df['Party'].str.lower()
# few fixes taken from https://www.kaggle.com/usman786/exploratory-data-analysis-for-interesting-insights/notebook
df['Party'].replace(['muttahida qaumi movement pakistan'], 'muttahida qaumi movement', inplace = True)
df['Party'].replace(['indeindependentdente','independent (retired)','indepndent'], 'independent',inplace = True)
df['Party'].replace(['indeindependentdente','independent (retired)','indepndent'], 'independent',inplace = True)
df['Party'].replace(['muttahidda majlis-e-amal pakistan','mutthida\xa0majlis-e-amal\xa0pakistan'
                     ,'mutthidaï¿½majlis-e-amalï¿½pakistan'] 
                     ,'muttahidda majlis-e-amal' ,inplace = True)
df['Party'].replace(['nazim-e-mistafa'], 'nizam-e-mustafa party' ,inplace = True)
df['Party'].replace(['pakistan muslim league (qa)'], 'pakistan muslim league (q)' ,inplace = True)
df['Party'].replace(['pakistan muslim league council'], 'pakistan muslim league (c)' ,inplace = True)
df['Party'].replace(['pakistan muslim league \x93h\x94 haqiqi'], 'pakistan muslim league haqiqi' ,inplace = True)
df['Party'].replace(['pakistan muslim league(z)'], 'pakistan muslim league (z)' ,inplace = True)
df['Party'].replace(['pakistan peoples party(shaheed bhutto)'], 'pakistan peoples party (shaheed bhutto)' ,inplace = True)
df['Party'].replace(['pakistan peoples party parliamentarians'], 'pakistan peoples party parliamentarians' ,inplace = True)
df['Party'].replace(['pakistan sariaki party'], 'Pakistan Siraiki Party (T)' ,inplace = True)
df['Party'].replace(['pasban'], 'pasban pakistan' ,inplace = True)
df['Party'].replace(['qaumi watan party (sherpao)'], 'qaumi watan party' ,inplace = True)
df['Party'].replace(['tehreek-e-suba hazara'], 'tehreek-e-suba hazara pakistan' ,inplace = True)
#...
df['Party'].replace(['pashtoonkhwa milli awami party'], 'pakhtoonkhwa milli Awami party' ,inplace = True)
df['Party'].replace(['pakistan amn party'], 'pakistan aman party' ,inplace = True)
df['Party'].replace(['pakistan awami inqelabi'], 'Pakistan Awami Inqelabi League' ,inplace = True)
df['Party'].replace(['pakistan freedom party'], 'pakistan freedom movement' ,inplace = True)
df['Party'].replace(['pakistan insani haqook party (pakistan human rights party)'], 'pakistan human rights party' ,inplace = True)
df['Party'].replace(['awami justice party'], 'awami justice party pakistan' ,inplace = True)
df['Party'].replace(['indeindependentdent'], 'independent' ,inplace = True)
df['Party'].replace(['jamiat ulama-e-pakistan  (noorani)'], 'jamiat ulama-e-pakistan (noorani)' ,inplace = True)
df['Party'].replace(['jumiat ulma-e-islam(nazryati)'], 'jamiat ulma-e-islam nazryati pakistan' ,inplace = True)
df['Party'].replace(['majlis-e-wahdat-e-muslimeen pakistan'], 'Majlis Wahdat-e-Muslimeen Pakistan' ,inplace = True)
df['Party'].replace(['markazi jamat-al-hadais'], 'Markazi Jamiat Ahl-e-Hadith' ,inplace = True)
df['Party'].replace(['mohib-e-wattan nowjawan inqilabion ki anjuman (mnaka)'], 'Muhib-e-Watan Noujawan Anqlabion Ki Anjuman (MNAKA)' ,inplace = True)

pty = df['Party'].unique()
pty.sort()
pty

array(['Majlis Wahdat-e-Muslimeen Pakistan',
       'Markazi Jamiat Ahl-e-Hadith',
       'Muhib-e-Watan Noujawan Anqlabion Ki Anjuman (MNAKA)',
       'Pakistan Siraiki Party (T)', 'aap janab sarkar party',
       'afghan qumi movement (pakistan)', 'all pakistan bayrozgar party',
       'all pakistan muslim league', 'all pakistan youth working party',
       'awami himayat tehreek pakistan', 'awami jamhuri ittehad pakistan',
       'awami justice party pakistan', 'awami muslim league pakistan',
       'awami national party', 'awami qaidat party', 'awami qiadat party',
       'awami workers party', 'azad pakistan party',
       'bahawalpur national awami party', 'balochistan national congress',
       'balochistan national democratic party',
       'balochistan national movement', 'balochistan national party',
       'balochistan national party (awami)',
       'christian progressive movement', 'communist party of pakistan',
       'ghareeb awam party', 'hazara awami ittehad pakistan',

In [43]:
#del pty
#convert textual content to lower case & remove trailing white spaces
df['CandidateName'] = df['CandidateName'].str.lower()
df['CandidateName'] = df['CandidateName'].str.strip()
df['CandidateName'].head(10)

0                     mr sajid abdullah
1                       mr shabir ahmad
2                mr usman bashir bilour
3    mr muhammad khurshid khan advocate
4              mr muhammad muazzam butt
5                 dr arbab alamgir khan
6     mr abdul manan akhunzada advocate
7                  maulana rehmat ullah
8            mr arbab muhammad ayub jan
9                 mr iqbal zafar jhagra
Name: CandidateName, dtype: object

We will remove Mr Initial from the begining of names, But we will keep Dr Initial because it is worth gaining title. 

In [44]:
# remove mr at the beginning of names.
df['CandidateName'] = df.loc[:, 'CandidateName'].replace(regex=True, to_replace="mr ", value="")
df['CandidateName'] = df.loc[:, 'CandidateName'].replace(regex=True, to_replace="mrs ", value="")
df['CandidateName'] = df.loc[:, 'CandidateName'].replace(regex=True, to_replace="miss ", value="")
#df['CandidateName'] = df.loc[:, 'CandidateName'].replace(regex=True, to_replace="mis ", value="")
df['CandidateName'].head(10)

0                     sajid abdullah
1                       shabir ahmad
2                usman bashir bilour
3    muhammad khurshid khan advocate
4              muhammad muazzam butt
5              dr arbab alamgir khan
6     abdul manan akhunzada advocate
7               maulana rehmat ullah
8            arbab muhammad ayub jan
9                 iqbal zafar jhagra
Name: CandidateName, dtype: object

In [45]:
cn = df['CandidateName'].unique()
cn.sort()
print("cn size: ", cn.shape, "\nValues: ", cn) 

cn size:  (7657,) 
Values:  ['aadil altaf unar' 'aamanullah' 'aamar sohail mughal' ... 'zulqurnain'
 'zumarad khan' 'zumurrad khan']


In [46]:
df['CandidateName']

0                        sajid abdullah
1                          shabir ahmad
2                   usman bashir bilour
3       muhammad khurshid khan advocate
4                 muhammad muazzam butt
                     ...               
4505                       gulab baloch
4506              muhammad yasin baloch
4507                         nazimuddin
4508                    sayed essa nori
4509                      zobaida jalal
Name: CandidateName, Length: 8619, dtype: object

Lets observe few to set the threshold for fuzzywuzzy

In [47]:
fuzzywuzzy.process.extract("zumurad khan", cn, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >90
fuzzywuzzy.process.extract("zobaida jalal", cn, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >79
#fuzzywuzzy.process.extract("barkat ali", cn, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >90
#fuzzywuzzy.process.extract("sher muhammad baloch", cn, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >90
#fuzzywuzzy.process.extract("gulab baloch", cn, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >90
#fuzzywuzzy.process.extract("babu gulab", cn, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio) # acceptance value >90

[('zobaida jalal', 100),
 ('zubada jalal', 88),
 ('zubeda jalal', 80),
 ('jalil zahid', 75),
 ('laiq zada', 64),
 ('qaisar jamal', 64),
 ('saida jan', 64),
 ('zahid ali', 64),
 ('umaid ali dal', 62),
 ('aqal zaman', 61)]

In [ ]:
replace_matches_in_column(df=df, column='CandidateName', string_to_match="zumurad khan", min_ratio=92)
replace_matches_in_column(df=df, column='CandidateName', string_to_match="zobaida jalal", min_ratio=80)
replace_matches_in_column(df=df, column='CandidateName', string_to_match="barkat ali", min_ratio=90)
replace_matches_in_column(df=df, column='CandidateName', string_to_match="muhammad yasin baloch", min_ratio=90)

for candi in df['CandidateName'].unique(): # 7000
    replace_matches_in_column(df=df, column='CandidateName', string_to_match=candi, min_ratio=90)

# let us know the loop is completed
print("All done!")

In [ ]:
#del NA2, NA8, NA13
df.to_csv('NA2002-18.csv', index=None) 

<h2>Candidate List & Parties of 2018 Election</h2>
Lets se if these 2 files need some cleaning as well. We will merge both files. 

In [49]:
cc = pd.read_csv("data/National Assembly Candidates List - 2018 Updated.csv", encoding = "ISO-8859-1")
na18 = pd.read_csv("data/2013-2018 Seat Changes in NA.csv", encoding = "ISO-8859-1") 
pp = pd.read_csv("data/Political Parties in 2018 Elections - Updated.csv", encoding = "ISO-8859-1")
print(cc.shape, na18.shape, pp.shape)

(3438, 4) (288, 3) (119, 3)


In [50]:
print(cc.columns, na18.columns)

Index(['NA#', 'Name', 'Party', 'Province'], dtype='object') Index(['2018 Seat Number', 'Seat Name', '2013 Seat Number'], dtype='object')


Adding "NA-" string in NA# column to merge it with na18 dataset.

In [51]:
cc['NA#'] = 'NA-' + cc['NA#'].astype(str)

In [52]:
print(cc['NA#'].unique().shape) # 272
print(na18['2018 Seat Number'].unique().shape) # 273
na18.rename(columns={'2018 Seat Number':'NA#'}, inplace=True)
na18.rename(columns={'Seat Name':'Seat'}, inplace=True)
na18[na18['NA#'] == "Old Constituency Changed Considerably"]

(272,)
(273,)


,NA#,Seat,2013 Seat Number
272,Old Constituency Changed Considerably,Karachi 9 - Central 5,NA-247
273,Old Constituency Changed Considerably,Karachi 12 - South 3,NA-250
274,Old Constituency Changed Considerably,Attock III,NA-059
275,Old Constituency Changed Considerably,Faisalabad 11,NA-085
276,Old Constituency Changed Considerably,Jhang- Cum-Chiniot (Old NA-87 Jhang-II),NA-088
277,Old Constituency Changed Considerably,Gujaranwala 7,NA-101
278,Old Constituency Changed Considerably,Hafizabad 2,NA-103
279,Old Constituency Changed Considerably,Narowal 3,NA-117
280,Old Constituency Changed Considerably,Nankana Sahib-III (Old Sheikhupra-VII),NA-137
281,Old Constituency Changed Considerably,Kasur 5,NA-142


In [53]:
na18 = na18[na18['NA#'] != "Old Constituency Changed Considerably"]
na18['NA#'] = na18.loc[:, 'NA#'].replace(regex=True, to_replace="NA-", value="")
na18['NA#'] = pd.to_numeric(na18['NA#'])
na18['NA#'] = na18['NA#'].astype(np.int64)
na18['NA#'] = 'NA-' + na18['NA#'].astype(str)
#na18['NA#'] = na18.loc[:, 'NA#'].replace(regex=True, to_replace=".0", value="")
na18['NA#'].head()

0    NA-1
1    NA-2
2    NA-3
3    NA-4
4    NA-5
Name: NA#, dtype: object

Lets add District column and do its cleaning

In [54]:
#Add District column & its cleani
na18['Distirct'] = na18['Seat']
# remove all those substring with () 
na18['Distirct'] = na18['Distirct'].str.replace(r"\(.*\)","")
# remove numeric
na18['Distirct']  = na18['Distirct'].str.replace('[^a-zA-Z -]', '')
na18['Distirct'] = na18['Distirct'].str.replace(r"Cum.*","")
#na18['Distirct'] = na18['Distirct'].str.replace(r"KUM.*","")
# to convert Tribal Area III - Mohman into Tribal Area III
na18['Distirct'] = na18['Distirct'].str.replace(r"-.*","")
na18['Distirct']  = na18['Distirct'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
na18['Distirct']  = na18['Distirct'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
na18['Distirct'].unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1439453734.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  na18['Distirct'] = na18['Distirct'].str.replace(r"\(.*\)","")
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1439453734.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  na18['Distirct']  = na18['Distirct'].str.replace('[^a-zA-Z -]', '')
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1439453734.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  na18['Distirct'] = na18['Distirct'].str.replace(r"Cum.*","")
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1439453734.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  na18['Distirct'] = na18['Distirct'].str.replace(r"-.*","")
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1439453734.py:11: FutureWarning: The default value of regex will ch

array(['Chitral', 'Swat', 'Upper Dir', 'Lower Dir', 'Malakand', 'Boner',
       'Shangla', 'Kohistan', 'Battagram', 'Mansehra', 'Abottabad',
       'Haripur', 'Swabi', 'Mardan', 'Charsadda', 'Nowshera', 'Peshawar',
       'Kohat', 'Hangu', 'Karak', 'Bannu', 'Lakki Marwat', 'Tank',
       'D I Khan', 'D I Khan KUM Tank', 'Tribal Area', 'Islamabad',
       'Attock', 'Rawalpindi', 'Chakwal', 'Jehlum', 'Gujarat', 'Sialkot',
       'Narowal', 'Gujaranwala', 'MANDI BAHAUDDIN', 'Hafizabad',
       'Sargodha', 'Khushab', 'Mianwali', 'Bhakkar', 'Chiniot',
       'Faisalabad', 'Toba Tek Singh', 'Jhang', 'Nankana Sahib',
       'Sheikhupura', 'Lahore', 'Kasur', 'Okara', 'Pakpattan', 'Sahiwal',
       'Khanewal', 'Multan', 'Lodhran', 'Vehari', 'Bahawalnagar',
       'Bahawalpur', 'Rahim Yar Khan', 'Muzaffargarh', 'Layyah',
       'Dera Ghazi Khan', 'Rajanpur', 'JACOBABAD', 'KASHMORE',
       'SHIKARPUR', 'SHEIKHUPUR', 'Larkana', 'LARKANA',
       'KAMBER SHAHDADKOT', 'GHOTKI', 'SUKKUR', 'Khairpur'

In [55]:
cc = cc.join(na18.set_index('NA#'), on='NA#')
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3438 entries, 0 to 3437
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NA#               3438 non-null   object
 1   Name              3435 non-null   object
 2   Party             3438 non-null   object
 3   Province          3438 non-null   object
 4   Seat              3438 non-null   object
 5   2013 Seat Number  3438 non-null   object
 6   Distirct          3438 non-null   object
dtypes: object(7)
memory usage: 188.1+ KB


In [56]:
print(pp.shape)
pp['Name of Political Party'].unique()

(119, 3)


array(['Aam Admi Tehreek Pakistan', 'Aam Awam Party',
       'Aam Log Party Pakistan',
       'All Pakistan Minority Movement Pakistan',
       'All Pakistan Muslim League',
       'All Pakistan Muslim League (Jinnah)', 'All Pakistan Tehreek',
       'Allah-O-Akbar Tehreek', 'Amun Taraqqi Party', 'Awam League',
       'Awami Justice Party Pakistan', 'Awami Muslim League Pakistan',
       'Awami National Party', 'Awami Party Pakistan-S',
       'Awami Workers Party', 'Balochistan Awami Party',
       'Balochistan National Movement', 'Balochistan National Party',
       'Balochistan National Party(Awami)', 'Barabri Party Pakistan',
       'Front National Pakistan', 'Grand Democratic Alliance',
       'Hazara Democratic Party', 'Humdardan-e-Watan Pakistan',
       'Islami Jamhoori Ittehad Pakistan', 'Islami Tehreek Pakistan',
       'Ittehad-e-Ummat Pakistan', 'Jamat-e-Islami Pakistan',
       'Jamhoori Watan Party', 'Jamiat Ulama-e-Islam Nazaryati Pakistan',
       'Jamiat Ulama-e-Islam(

In [57]:
pp.rename(columns={'Acronym':'PartyAcro'}, inplace=True)
cc.rename(columns={'Party':'PartyAcro'}, inplace=True)
pp.rename(columns={'Name of Political Party':'Party'}, inplace=True)

In [58]:
# Clean Candidate file
pp['Party'].replace(['pakistan reh-e- haq party'], 'Pakistan Rah-e- Haq Party' ,inplace = True)
pp['Party'].replace(['Pakistan Muslim League SHER-E-BANGAL A.K. Fazal-Ul-Haque'], 'pakistan muslim league (sher-e-bangal)' ,inplace = True)
pp['Party'].replace(['Pakistan Muslim League (Zia-ul-Haq Shaheed)'], 'pakistan muslim league (z)' ,inplace = True)
pp['Party'].replace(['Pakistan Muslim League (Junejo)'], 'pakistan muslim league (j)' ,inplace = True)
pp['Party'].replace(['Pakistan Muslim League (Functional)'], 'pakistan muslim league (f)' ,inplace = True)
pp['Party'].replace(['Pakistan Muslim League (Council)'], 'pakistan muslim league (c)' ,inplace = True)
pp['Party'].replace(['Pakistan Muslim League-Nawaz'], 'pakistan muslim league (n)' ,inplace = True)
pp['Party'].replace(['Pakistan Justice & Democratic Party'], 'Pakistan Justice and Democratic Party' ,inplace = True)
pp['Party'].replace(['Pakistan Kissan Ittehad (Ch. Anwar)'], 'Pakistan Kissan Ittehad' ,inplace = True)
pp['Party'].replace(['Jamiat Ulma-e-Islam Nazryati Pakistan'], 'Jamiat Ulma-e-Islam Nazaryati Pakistan' ,inplace = True)
pp['Party'].replace(['Jamiat Ulma-e-Islam Nazryati Pakistan'], 'Jamiat Ulma-e-Islam Nazaryati Pakistan' ,inplace = True)
pp['Party'].replace(['Jamiat Ulama-e-Islam(F)'], 'Jamiat Ulama-e-Islam (F)' ,inplace = True)
pp['Party'].replace(['Jamiat Ulama-e-Islam(S)'], 'Jamiat Ulama-e-Islam (S)' ,inplace = True)
pp['Party'].replace(['Mohajir Qaumi Movement (Pakistan)'], 'Mohajir Qaumi Movement pakistan' ,inplace = True)
pp['Party'].replace(['Mutahida Majlis-e-Amal'], 'Muttahida Majlis-e-Amal' ,inplace = True)
pp['Party'].replace(['Muttahidda Qaumi Movement Pakistan'], 'Muttahida Qaumi Movement Pakistan' ,inplace = True)


In [59]:
# Remove duplicaties
pp.drop_duplicates(subset=['PartyAcro'], keep="first", inplace=True)
pp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 0 to 118
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Party      112 non-null    object
 1   Symbol     106 non-null    object
 2   PartyAcro  111 non-null    object
dtypes: object(3)
memory usage: 3.5+ KB


In [60]:
pp

,Party,Symbol,PartyAcro
0,Aam Admi Tehreek Pakistan,Mug,AATP
1,Aam Awam Party,Wheat Bunch,AAP
2,Aam Log Party Pakistan,Hut,ALPP
3,All Pakistan Minority Movement Pakistan,Giraffe,APMMP
4,All Pakistan Muslim League,Eagle,APML
...,...,...,...
111,Pakistan Muslim League (Q),cycle,PMLQ
112,Pakistan Justice and Democratic Party,NaN,PJDP
113,Justice and Development Party Pakistan,NaN,JDPP
117,Tehreek Jamhuriat Pakistan,NaN,TJ


In [61]:
cc[cc['PartyAcro']=='PTI'].head()
#pp[pp['PartyAcro']=='PTEI']

,NA#,Name,PartyAcro,Province,Seat,2013 Seat Number,Distirct
10,NA-1,Abdul Latif,PTI,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral
20,NA-2,Dr. Haider Ali Khan,PTI,Khyber Pakhtunkhwa,Swat I,NA-029,Swat
28,NA-3,Saleem Rehman,PTI,Khyber Pakhtunkhwa,Swat II,NA-030,Swat
37,NA-4,Murad Saeed,PTI,Khyber Pakhtunkhwa,Swat III,Newly Added,Swat
49,NA-5,Sahibzada Sibghat Ullah,PTI,Khyber Pakhtunkhwa,Upper Dir,NA-033,Upper Dir


In [62]:
#del cnd
cnd = cc.join(pp.set_index('PartyAcro'), on='PartyAcro')

In [63]:
cnd.info()
cnd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3438 entries, 0 to 3437
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NA#               3438 non-null   object
 1   Name              3435 non-null   object
 2   PartyAcro         3438 non-null   object
 3   Province          3438 non-null   object
 4   Seat              3438 non-null   object
 5   2013 Seat Number  3438 non-null   object
 6   Distirct          3438 non-null   object
 7   Party             1706 non-null   object
 8   Symbol            1686 non-null   object
dtypes: object(9)
memory usage: 241.9+ KB


,NA#,Name,PartyAcro,Province,Seat,2013 Seat Number,Distirct,Party,Symbol
0,NA-1,Eid Ul Hussain,ANP,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral,Awami National Party,Lantern
1,NA-1,Mohammad Amjad,APML,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral,All Pakistan Muslim League,Eagle
2,NA-1,Mohammad Yahya,Independent,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral,NaN,NaN
3,NA-1,Nisar Dastageer,Independent,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral,NaN,NaN
4,NA-1,Shahzada Muharamad Taim,Independent,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral,NaN,NaN


In [64]:
cnd[cnd['PartyAcro']=="PTI"].head()
#remove non-aplhabetic characters from Name
cnd['Name'] = cnd['Name'].str.replace('[^a-zA-Z ]', '')
cnd['Name'] = cnd['Name'].str.lower()
cnd['Name'] = cnd['Name'].str.strip()

cnd['Party'] = cnd['Party'].str.lower()
cnd['Party'] = cnd['Party'].str.strip()

cnd[cnd['PartyAcro']=="PTI"].head()

C:\Users\HP\AppData\Local\Temp\ipykernel_8976\1262647537.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  cnd['Name'] = cnd['Name'].str.replace('[^a-zA-Z ]', '')


,NA#,Name,PartyAcro,Province,Seat,2013 Seat Number,Distirct,Party,Symbol
10,NA-1,abdul latif,PTI,Khyber Pakhtunkhwa,Chitral,NA-032,Chitral,pakistan tehreek-e-insaf,Bat
20,NA-2,dr haider ali khan,PTI,Khyber Pakhtunkhwa,Swat I,NA-029,Swat,pakistan tehreek-e-insaf,Bat
28,NA-3,saleem rehman,PTI,Khyber Pakhtunkhwa,Swat II,NA-030,Swat,pakistan tehreek-e-insaf,Bat
37,NA-4,murad saeed,PTI,Khyber Pakhtunkhwa,Swat III,Newly Added,Swat,pakistan tehreek-e-insaf,Bat
49,NA-5,sahibzada sibghat ullah,PTI,Khyber Pakhtunkhwa,Upper Dir,NA-033,Upper Dir,pakistan tehreek-e-insaf,Bat


Merging .. 

In [65]:
print(df.columns, cnd.columns)
df.info()
cnd.info()

Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout', 'Year'],
      dtype='object') Index(['NA#', 'Name', 'PartyAcro', 'Province', 'Seat', '2013 Seat Number',
       'Distirct', 'Party', 'Symbol'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8619 entries, 0 to 4509
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   District               6425 non-null   object 
 1   Seat                   8619 non-null   object 
 2   ConstituencyTitle      8619 non-null   object 
 3   CandidateName          8619 non-null   object 
 4   Party                  8619 non-null   object 
 5   Votes                  8619 non-null   int64  
 6   TotalValidVotes        8619 non-null   int64  
 7   TotalRejectedVotes     8619 non-null   int64  
 8   TotalVotes   

In [66]:
cnd.rename(columns={'NA#':'ConstituencyTitle'}, inplace=True)
cnd.rename(columns={'Name of Political Party':'Party'}, inplace=True)
cnd.rename(columns={'Name':'CandidateName'}, inplace=True)

In [67]:
cnd.to_csv('Canditates2018.csv', index=None) 
pp.to_csv('Parties_cleand.csv', index=None)

<h3><u>Note: Both files can not is mergered easily as NA mapping is changed for current year.  </u></h3>
I will use both files in EDA and Feature Engineering.

That's all from me. I tried to clean maximum of the data inconsistency issues. So, I am saving this file for the audience for the seek a reusability. You can fork kernel and continue from here.

We are all set to move towards <b>Exploratory Data Analysis </b>. 

Do share your comments and if you find it helpful, <b>please upvote! </b>
<h2> Happy Exploratory Analsysis :-) </h2>

<h2> Data Cleaning of NA 2018 Election Results <h2>

In [68]:

# Reading 2018 Results Data
NA18 = pd.read_csv("data/NA-Results2018 Ver 2.csv", encoding = "ISO-8859-1")
print("Data Dimensions of NA18 are: ", NA18.shape)

print("\nNA 2018.csv")
NA18.info()

NA18 = NA18.drop('Unnamed: 0', axis=1)
NA18.rename(columns={'district':'District'}, inplace=True)

NA18.District = NA18.Seat
NA18['District'] = NA18['District'].str.replace("."," ") # to deal with D.I. Khan
NA18['District'] = NA18['District'].str.replace(r"\(.*\)","")
NA18['District']  = NA18['District'] .str.replace('[^a-zA-Z -]', '')
NA18['District'] = NA18['District'].str.replace(r"-.*","")
NA18['District']  = NA18['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA18['District']  = NA18['District'] .str.replace(r" (XX|IX|X?I{0,3})(IX|IV|V?I{0,3})$", '')
NA18['District'].unique()

NA18['Turnout'] = NA18['Turnout'].str.rstrip('%').str.rstrip(' ')
NA18['Turnout'] = pd.to_numeric(NA18['Turnout'], errors='coerce')
NA18.rename(columns={'Constituency_Title':'ConstituencyTitle', 'Candidate_Name':'CandidateName', 'Total_Valid_Votes':'TotalValidVotes', 'Total_Rejected_Votes':'TotalRejectedVotes', 'Total_Votes':'TotalVotes', 'Total_Registered_Voters':'TotalRegisteredVoters', 'Part':'Party' }, inplace=True)
NA18.columns

Data Dimensions of NA18 are:  (3428, 12)

NA 2018.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3428 entries, 0 to 3427
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               3428 non-null   int64 
 1   district                 3428 non-null   object
 2   Seat                     3428 non-null   object
 3   Constituency_Title       3428 non-null   object
 4   Candidate_Name           3428 non-null   object
 5   Part                     3428 non-null   object
 6   Votes                    3428 non-null   int64 
 7   Total_Valid_Votes        3428 non-null   int64 
 8   Total_Rejected_Votes     3428 non-null   int64 
 9   Total_Votes              3428 non-null   int64 
 10  Total_Registered_Voters  3428 non-null   int64 
 11  Turnout                  3428 non-null   object
dtypes: int64(6), object(6)
memory usage: 321.5+ KB


C:\Users\HP\AppData\Local\Temp\ipykernel_8976\515740270.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NA18['District'] = NA18['District'].str.replace("."," ") # to deal with D.I. Khan
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\515740270.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  NA18['District'] = NA18['District'].str.replace(r"\(.*\)","")
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\515740270.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  NA18['District']  = NA18['District'] .str.replace('[^a-zA-Z -]', '')
C:\Users\HP\AppData\Local\Temp\ipykernel_8976\515740270.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  NA18['District'] = NA18['District'].str.rep

Index(['District', 'Seat', 'ConstituencyTitle', 'CandidateName', 'Party',
       'Votes', 'TotalValidVotes', 'TotalRejectedVotes', 'TotalVotes',
       'TotalRegisteredVoters', 'Turnout'],
      dtype='object')

In [69]:
# convert to lower case
NA18['District'] = NA18['District'].str.lower()
# remove trailing white spaces
NA18['District'] = NA18['District'].str.strip()

# convert to lower case
NA18['CandidateName'] = NA18['CandidateName'].str.lower()
# remove trailing white spaces
NA18['CandidateName'] = NA18['CandidateName'].str.strip()

# convert to lower case
NA18['Party'] = NA18['Party'].str.lower()
# remove trailing white spaces
NA18['Party'] = NA18['Party'].str.strip()

In [70]:
NA18.head()

,District,Seat,ConstituencyTitle,CandidateName,Party,Votes,TotalValidVotes,TotalRejectedVotes,TotalVotes,TotalRegisteredVoters,Turnout
0,chitral,Chitral,NA-1,moulana abdul akbar chitrali,muttahida majlis-e-amal pakistan,48616,158925,5430,164355,269579,60.97
1,chitral,Chitral,NA-1,saeed ur rehman,pakistan rah-e-haq party,3223,0,0,0,0,60.97
2,chitral,Chitral,NA-1,muhammad yahya,independent,698,0,0,0,0,60.97
3,chitral,Chitral,NA-1,shahzada muhammad taimur khisrao,independent,2414,0,0,0,0,60.97
4,chitral,Chitral,NA-1,eid ul hussain,awami national party,3613,0,0,0,0,60.97


In [71]:
NA18.to_csv('NA2018_Clean.csv', index=None)